In [10]:
import pandas as pd
import numpy as np
import sys
import os
import warnings

warnings.filterwarnings('ignore') # Opcional: para limpar os logs

# --- Adiciona a pasta 'src' ao path do Python ---
project_root = os.path.abspath(os.path.join('..'))

# CORREÇÃO: O seu código .py está na pasta 'src', não 'data'
data_path = os.path.join(project_root, 'data')

# Usamos insert(0, ...) para dar prioridade à pasta 'src'
if data_path not in sys.path:
    sys.path.insert(0, data_path) 
# -----------------------------------------------

# ADIÇÃO: Fazemos os imports aqui na Célula 1
# Agora o Python sabe onde encontrar 'data.make_dataset' (dentro de 'src')
from data.make_dataset import get_historical_data, get_metas_from_redshift

print("Módulos e funções importados com sucesso.")
print(f"Raiz do projeto: {project_root}")
print(f"Pasta 'src' priorizada no path.")

ModuleNotFoundError: No module named 'data'

In [ ]:
print("--- FASE 1: Coleta de Dados ---")

# 1. Busca o histórico de transações (TPV, Margem)
# (Usando a função que já tínhamos)
df_historico = get_historical_data(dat_start_filter='2024-01-01') # Use um período maior

# 2. Busca as Metas de TPV
# (Usando a nova função que busca do Redshift)
df_metas = get_metas_from_redshift()

# 3. Verificações
if df_historico is None or df_historico.empty:
    print("\n--- ERRO CRÍTICO (FASE 1) ---")
    print("A função get_historical_data() falhou ou não retornou dados.")
    print("Verifique a conexão e a query em src/data/make_dataset.py")
elif df_metas is None or df_metas.empty:
    print("\n--- ERRO CRÍTICO (FASE 1) ---")
    print("A função get_metas_from_redshift() falhou ou não retornou dados.")
    print("Verifique a query de metas em src/data/make_dataset.py")
else:
    print("\n--- SUCESSO (FASE 1) ---")
    print(f"{len(df_historico)} registros históricos carregados.")
    print(f"{len(df_metas)} metas de clientes carregadas.")
    
    print("\nAmostra dos dados históricos:")
    print(df_historico.head())
    
    print("\nAmostra das metas:")
    print(df_metas.head())

--- FASE 1: Coleta de Dados ---


NameError: name 'get_historical_data' is not defined

In [ ]:
print("\n--- FASE 2: Engenharia de Atributos ---")

# Só executa se ambas as coletas da FASE 1 funcionarem
if (df_historico is not None) and (df_metas is not None):
    
    # Executa a função principal da FASE 2
    # Passamos os dados brutos e as metas
    df_features = engineer_features(df_historico, df_metas)
    
    # --- RESULTADO FINAL ---
    print("\n--- SUCESSO (FASE 2) ---")
    print(f"Tabela final criada com {df_features.shape[0]} linhas (clientes) e {df_features.shape[1]} colunas (features).")
    
    # Mostra o resultado
    print("\nAmostra do DataFrame de Features (Resultado):")
    print(df_features.head())
    
    # Mostra as colunas criadas
    print("\nColunas criadas:")
    print(list(df_features.columns))
    
    # Salva o DataFrame de features para usar na próxima fase (Treinamento)
    # O '..' volta para a raiz do projeto
    output_dir = os.path.join(project_root, 'dados', 'processed')
    output_path = os.path.join(output_dir, 'features_clientes.csv')
    
    # Garante que a pasta /dados/processed exista
    os.makedirs(output_dir, exist_ok=True)
    
    df_features.to_csv(output_path)
    print(f"\nDataFrame de features salvo em: {output_path}")

else:
    print("\n--- FALHA (FASE 2) ---")
    print("A engenharia de atributos foi pulada devido a falhas na FASE 1.")